In [114]:
import pandas as pd
import numpy as np
import os
import sys

### Read the data

In [115]:
current_path = os.getcwd()
day = int(current_path.split('day')[1])

fn = 'day' + str(day) + '.txt'

file_content = open(fn).read().split('\n')

test_fn = 'day' + str(day) + 'test.txt'

test_file_content = open(test_fn).read().split('\n')

### Part 1

In [116]:
def get_array_data(which_file_content):
    working_file = which_file_content.copy()
    working_file_array = list()
    for i in range(len(working_file)):
        # convert string to list of characters
        row_list = list(working_file[i])
        # convert 'O' to '1` and '.' to 0 and '#' to 2
        for j in range(len(row_list)):
            if row_list[j] == 'O':
                row_list[j] = 1
            elif row_list[j] == '.':
                row_list[j] = 0
            elif row_list[j] == '#':
                row_list[j] = 2
        working_file_array.append(row_list)

    # convert to numpy array
    working_file_array = np.array(working_file_array)
    return working_file_array


In [117]:
working_file_array = get_array_data(file_content)

for i, row in enumerate(working_file_array):
    for j, char in enumerate(row):
        if char == 1:
            last_available_i = i
            for k in range(i-1, -2, -1):
                if k < 0 or working_file_array[k][j] in [1, 2]:
                    if last_available_i == i:
                        break
                    working_file_array[last_available_i][j] = 1
                    working_file_array[i][j] = 0
                    break
                last_available_i = k
                
load = 0

for i, row in enumerate(working_file_array):
    load += np.sum(row == 1) * (len(working_file_array) - i)

print(load)


108889


### Part 2

In [327]:
def spin_cycle(working_file_array):
    for direction in ['north', 'west', 'south', 'east']:
        if direction == 'south':
            # Flip upside down
            working_file_array = np.flipud(working_file_array)
        elif direction == 'east':
            # Flip left to right
            working_file_array = np.fliplr(working_file_array)
        
        for i, row in enumerate(working_file_array):
            for j, char in enumerate(row):
                if char == 1:
                    if direction in ['north', 'south']:
                        last_available_i = i
                        for k in range(i-1, -2, -1):
                            if k < 0 or working_file_array[k][j] in [1, 2]:
                                if last_available_i == i:
                                    break
                                working_file_array[last_available_i][j] = 1
                                working_file_array[i][j] = 0
                                break
                            last_available_i = k
                    elif direction in ['east', 'west']:
                        last_available_j = j
                        for k in range(j-1, -2, -1):
                            if k < 0 or working_file_array[i][k] in [1, 2]:
                                if last_available_j == j:
                                    break
                                working_file_array[i][last_available_j] = 1
                                working_file_array[i][j] = 0
                                break
                            last_available_j = k
        if direction == 'south':
            # Flip upside down
            working_file_array = np.flipud(working_file_array)
        elif direction == 'east':
            # Flip left to right
            working_file_array = np.fliplr(working_file_array)
    return working_file_array


class Cycle:
    def __init__(self, working_file_array, cycle_number):
        self.working_file_array = working_file_array.copy()
        self.cycle_number = cycle_number

    @property
    def load(self):
        load = 0
        for i, row in enumerate(self.working_file_array):
            load += np.sum(row == 1) * (len(self.working_file_array) - i)
        return(load)
    
    def __eq__(self, other):
        # check if the arrays are the same in every position
        return np.array_equal(self.working_file_array, other.working_file_array)

cache = list()
working_file_array = get_array_data(file_content)
i = 0
while True:
    working_file_array = spin_cycle(working_file_array)
    cycle = Cycle(working_file_array, i)
    if cycle in cache:
        cycle_length = i - cache.index(cycle)
        start_cycle = cache.index(cycle)
        x = 1000000000 - 1 - start_cycle

        earliest_same_cycle_as_x = x % cycle_length + start_cycle

        print(cache[earliest_same_cycle_as_x].load)

        break

    else:
        cache.append(cycle)
    i += 1


104671
